In [361]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window
import talib

In [362]:
global DoEval #是否進行模型評估

# DoEval = False
# next_date_range = ["20180611", "20180612", "20180613", "20180614", "20180615"] #設定預測區間
next_date_range = ["20180604", "20180605", "20180606", "20180607", "20180608"] #設定預測區間
# next_date_range = ["20180528", "20180529", "20180530", "20180531", "20180601"] #設定預測區間
# next_date_range = ["20180521", "20180522", "20180523", "20180524", "20180525"] #設定預測區間
DoEval = True
# next_date_range = ["20180514", "20180515", "20180516", "20180517", "20180518"] #設定預測區間
ignore_dates = [""]#設定排除日(如端午節)
# next_date_range = ["20180507", "20180508", "20180509", "20180510", "20180511"] #設定預測區間
predict_start_date = next_date_range[0]

if sc.master[0:5]=="local":
#     Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180601"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180511"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"
else:
    Path = "hdfs://master:9000/user/hduser"
Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180608"

In [363]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [364]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [365]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19575
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [366]:
#取出etf的distinct id
etf_ids = []
etf_idx_dic = {}
etf_idx = 0
for row in tetfp_dt.select("etf_id").distinct().orderBy("etf_id").collect():
    etf_ids.append(row["etf_id"])
    etf_idx_dic.update({row["etf_id"]: etf_idx})
    etf_idx += 1
etf_idx_dic
# etf_ids = ['0050   ']

{'0050   ': 0,
 '0051   ': 1,
 '0052   ': 2,
 '0053   ': 3,
 '0054   ': 4,
 '0055   ': 5,
 '0056   ': 6,
 '0057   ': 7,
 '0058   ': 8,
 '0059   ': 9,
 '006201 ': 10,
 '006203 ': 11,
 '006204 ': 12,
 '006208 ': 13,
 '00690  ': 14,
 '00692  ': 15,
 '00701  ': 16,
 '00713  ': 17}

In [367]:
etf_dic = {}
for etfid in etf_ids:
    export_dt = tetfp_dt.filter("etf_id='" +etfid+ "' and etf_date < '" + predict_start_date + "'") \
        .orderBy("etf_id", "etf_date", ascending=True)
    export_pd = export_dt.toPandas()
    etf_dic.update({etfid.strip(): export_pd})

In [368]:
#functions of TA lib
#print(talib.get_functions())

In [369]:
etf_dic["0050"]["etf_close"].count()

1327

In [370]:
import math
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)

def get_feature_pre(curr_list):
    #在feature前加上一個Nan後回傳, 第二個值則回傳原本的最後一個值作為下次的feature
    if type(curr_list) is pd.Series:
        rtn_list = curr_list.tolist()
    else:
        rtn_list = curr_list
    rtn_list = [None] + rtn_list
    return (rtn_list[:-1], [rtn_list[-1]])

def checkNan(num):
    if num == None:
        return True
    elif math.isnan(num):
        return True
    else:
        return False
    
def calculate_n_bias(close_price, ma):
#     (close price - MA)/MA
    if checkNan(close_price) or checkNan(ma):
        return None
    else:
        return (close_price - ma)/ma
    
def calculate_n_bias_list(close_ser, ma_list):
    #calculate n bias (20 days)
    bias_list = []
    for cprice, ma in zip(close_ser.tolist(), ma_list):
        bias_list.append(calculate_n_bias(cprice, ma))
    return bias_list

def calculate_bias_ma_list(ma3_list, ma6_list):
    #calculate bias_ma = (ma3-ma6)/ma6
    bias_list = []
    for ma3, ma6 in zip(ma3_list, ma6_list):
        bias_list.append(calculate_n_bias(ma3, ma6))
    return bias_list

def calculate_bias_3_6_list(bias3_list, bias6_list):
    #bias_3_6 = bias3 - bias6
    bias_list = []
    for bias3, bias6 in zip(bias3_list, bias6_list):
        if checkNan(bias3) or checkNan(bias6):
            bias_list.append(None)
        else:
            bias_list.append(bias3 - bias6)
    return bias_list

def calculate_dif(ema12_list, ema26_list):
    #差離值DIF = 12日EMA - 26日EMA 
    dif_list = []
    for ema12, ema26 in zip(ema12_list, ema26_list):
        if checkNan(ema12) or checkNan(ema26):
            dif_list.append(None)
        else:
            dif_list.append(ema12 - ema26)
    return dif_list
def calculate_slope(cur_list, prev_list):
    slope_list = []
    for cur, prev in zip(cur_list, prev_list):
        if checkNan(cur) or checkNan(prev):
            slope_list.append(None)
        else:
            slope_list.append((cur-prev)/prev)
    return slope_list

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import itertools
def create_feature(etf_pd, next_date, scaler_dic):
    close_ser = etf_pd["etf_close"]
    #EMA
    ema3 = talib.EMA(close_ser,timeperiod=3)
    ema3_prev, _ = get_feature_pre(ema3)
    ema3_slope = calculate_slope(ema3, ema3_prev)
    ema5 = talib.EMA(close_ser,timeperiod=5)
    ema5_prev, _ = get_feature_pre(ema5)
    ema5_slope = calculate_slope(ema5, ema5_prev)
    ema10 = talib.EMA(close_ser,timeperiod=10)
    ema10_prev, _ = get_feature_pre(ema10)
    ema10_slope = calculate_slope(ema10, ema10_prev)
    ema20 = talib.EMA(close_ser,timeperiod=20)
    ema30 = talib.EMA(close_ser,timeperiod=30)
    
    #make diff
    close_ser_prev, _ = get_feature_pre(close_ser)
    close_diff = calculate_dif(close_ser.tolist(), close_ser_prev)
    
    close_diff_2, _ = get_feature_pre(close_diff)
    close_diff_3, _ = get_feature_pre(close_diff_2)
    #BIAS
    #nBIAS -3, 6, 20 => (close price - MA)/MA   ,Paper 建議用20日MA
    ma3 = talib.MA(np.array(close_ser), timeperiod=3)
    ma6 = talib.MA(np.array(close_ser), timeperiod=6)
    ma20 = talib.MA(np.array(close_ser), timeperiod=20)
    nbias3 = calculate_n_bias_list(close_ser, ma3)
    nbias6 = calculate_n_bias_list(close_ser, ma6)
    nbias20 = calculate_n_bias_list(close_ser, ma20)
    
    #BIAS_ma = (ma3-ma6)/ma6
    bias_ma = calculate_bias_ma_list(ma3, ma6)
    
    #bias_3_6 = bias3 - bias6
    bias_3_6 = calculate_bias_3_6_list(nbias3, nbias6)
    
    #KD --> only use STOCHRSI
    k, d = talib.STOCHRSI(close_ser, timeperiod=9, fastk_period=3, fastd_period=3, fastd_matype=0)
    
    #差離值DIF = 12日EMA - 26日EMA 
    ema12 = talib.EMA(close_ser,timeperiod=12)
    ema26 = talib.EMA(close_ser,timeperiod=26)
    dif = calculate_dif(ema12, ema26)
    
    #MACD
#     macd1, macdsignal1, macdhist1 = talib.MACD(close_ser, fastperiod=12, slowperiod=26, signalperiod=9)
    macd1, macdsignal1, macdhist1 = talib.MACDEXT(close_ser, fastperiod=12, fastmatype=0, 
                                               slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
#     macd1, macdsignal1, macdhist1 = talib.MACDFIX(close_ser, signalperiod=9)
    #RSI
    rsi = talib.RSI(close_ser, timeperiod=10)
    
    #CMO
    cmo = talib.CMO(close_ser, timeperiod=14)
    
    #ROCR - Rate of change ratio: (price/prevPrice)
    #rocr = talib.ROCR(close_ser, timeperiod=10)
    #ROC - Rate of change : ((price/prevPrice)-1)*100
    rocr = talib.ROC(close_ser, timeperiod=10)
    
    #PPO - Percentage Price Oscillator
    ppo = talib.PPO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    #APO - Absolute Price Oscillator
    apo = talib.APO(close_ser, fastperiod=12, slowperiod=26, matype=0)
    
    #MOM - Momentum
    mom = talib.MOM(close_ser, timeperiod=10)
    #TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
    trix = talib.TRIX(close_ser, timeperiod=30)
    #作標準化
    etfid = etf_pd["etf_id"].tolist()[0]
    tmp_pd = pd.DataFrame({"etf_id": etfid, "etf_date": etf_pd["etf_date"], "etf_close": close_ser, "close_dif": close_diff,  
#                            "close_diff_1": close_diff, "close_diff_2": close_diff_2, "close_diff_3": close_diff_3,
                           "ema3":ema3, "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                           "ema3_slope": ema3_slope, "ema5_slope": ema5_slope, "ema10_slope": ema10_slope,
                          "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          #"macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                          "macd1": macd1, "macdsignal1": macdsignal1, "macdhist1": macdhist1,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo}).dropna(how='any')
    tmp_pd = tmp_pd[tmp_pd['etf_date'] > '20140101']
    #exclude outlier
#     for col in [c for c in tmp_pd.columns.values if c not in ["etf_id", "etf_date"]]:
#         tmp_pd = tmp_pd[np.abs(tmp_pd[col] - tmp_pd[col].mean()) <= 3.0*tmp_pd[col].std()] #只取3倍以內標準差資料
    
    std_dic = {}
    for col in [c for c in tmp_pd.columns.values if c not in ["etf_id", "etf_date"]]:
#         scaler_key = etfid.strip() + '-' + col
        scaler_key = col
        if scaler_key in scaler_dic:
            scaler = scaler_dic[scaler_key]
            merged = list(itertools.chain.from_iterable(scaler.transform(tmp_pd[[col]])))                
#             merged = tmp_pd[col]
        else:
            scaler = MinMaxScaler()
            merged = list(itertools.chain.from_iterable(scaler.fit_transform(tmp_pd[[col]])))
            merged = tmp_pd[col]
            #保留etf_close的scaler在後續使用
            scaler_dic.update({scaler_key: scaler})
        std_dic.update({col: merged})
    
    #作出目前的feature table及預測用的feature table
    ema3, next_ema3 = get_feature_pre(std_dic["ema3"])
    ema3_slope, next_ema3_slope = get_feature_pre(std_dic["ema3_slope"])
    ema5, next_ema5 = get_feature_pre(std_dic["ema5"])
    ema5_slope, next_ema5_slope = get_feature_pre(std_dic["ema5_slope"])
    ema10, next_ema10 = get_feature_pre(std_dic["ema10"])
    ema10_slope, next_ema10_slope = get_feature_pre(std_dic["ema10_slope"])
    ema20, next_ema20 = get_feature_pre(std_dic["ema20"])
    ema30, next_ema30 = get_feature_pre(std_dic["ema30"])
    
    nbias3, next_nbias3 = get_feature_pre(std_dic["nbias3"])
    nbias6, next_nbias6 = get_feature_pre(std_dic["nbias6"])
    nbias20, next_nbias20 = get_feature_pre(std_dic["nbias20"])
    
    bias_ma, next_bias_ma = get_feature_pre(std_dic["bias_ma"])
    bias_3_6, next_bias_3_6 = get_feature_pre(std_dic["bias_3_6"])
    k, next_k = get_feature_pre(std_dic['k'])
    d, next_d = get_feature_pre(std_dic['d'])
    dif, next_dif = get_feature_pre(std_dic['dif'])
    
    #macd, next_macd = get_feature_pre(std_dic['macd'])
    #macdsignal, next_macdsignal = get_feature_pre(std_dic['macdsignal'])
    #macdhist, next_macdhist = get_feature_pre(std_dic['macdhist'])
    macd1, next_macd1 = get_feature_pre(std_dic['macd1'])
    macdsignal1, next_macdsignal1 = get_feature_pre(std_dic['macdsignal1'])
    macdhist1, next_macdhist1 = get_feature_pre(std_dic['macdhist1'])
                                               
    rsi, next_rsi = get_feature_pre(std_dic['rsi'])
    cmo, next_cmo = get_feature_pre(std_dic['cmo'])
    rocr, next_rocr = get_feature_pre(std_dic['rocr'])
    ppo, next_ppo = get_feature_pre(std_dic['ppo'])
    
#     close_diff_1, next_close_diff_1 = get_feature_pre(std_dic['close_diff_1'])
#     close_diff_2, next_close_diff_2 = get_feature_pre(std_dic['close_diff_2'])
#     close_diff_3, next_close_diff_3 = get_feature_pre(std_dic['close_diff_3'])
    
    #"apo": apo, "mom": mom, "trix": trix
#     apo, next_apo = get_feature_pre(std_dic['apo'])
#     mom, next_mom = get_feature_pre(std_dic['mom'])
#     trix, next_trix = get_feature_pre(std_dic['trix'])
#     print(len(std_dic['etf_close']), ' ', len(close_diff_1), ' ', len(close_diff_2), ' ' , len(rsi))
    
    rtn_pd = pd.DataFrame({"etf_id": etfid, "idx": etf_idx_dic[etfid], "etf_date": tmp_pd["etf_date"], 
                           "etf_close": std_dic["etf_close"], "close_dif": std_dic['close_dif'], 
                           'ema3': ema3, "ema5": ema5, "ema10": ema10, "ema20": ema20, "ema30" : ema30,
                          "ema3_slope": ema3_slope, "ema5_slope": ema5_slope, "ema10_slope": ema10_slope,
                           "nbias3": nbias3, "nbias6": nbias6, "nbias20": nbias20, "bias_ma": bias_ma,
                          "bias_3_6": bias_3_6, "k": k, "d": d, "dif": dif,
                          #"macd": macd, "macdsignal": macdsignal, "macdhist": macdhist,
                           "macd1": macd1, "macdsignal1": macdsignal1, "macdhist1": macdhist1,
                          "rsi": rsi, "cmo": cmo, "rocr": rocr, "ppo": ppo}).dropna(how='any')
    #col = 'close_dif'
    #rtn_pd = rtn_pd[np.abs(rtn_pd[col] - rtn_pd[col].mean()) <= 3.0*rtn_pd[col].std()] #只取3倍以內標準差資料
    
    next_pd = pd.DataFrame({"etf_id": etfid, "idx": etf_idx_dic[etfid], "etf_date": next_date, 
                            'etf_close': -1.0, "close_dif": -1.0, 
                           "ema3": next_ema3, "ema5": next_ema5, "ema10": next_ema10, "ema20": next_ema20, "ema30" : next_ema30,
                          "ema3_slope": next_ema3_slope, "ema5_slope": next_ema5_slope, "ema10_slope": next_ema10_slope,
                            "nbias3": next_nbias3, "nbias6": next_nbias6, "nbias20": next_nbias20, "bias_ma": next_bias_ma,
                          "bias_3_6": next_bias_3_6, "k": next_k, "d": next_d, "dif": next_dif,
                          #"macd": next_macd, "macdsignal": next_macdsignal, "macdhist": next_macdhist,
                            "macd1": next_macd1, "macdsignal1": next_macdsignal1, "macdhist1": next_macdhist1,
                          "rsi": next_rsi, "cmo": next_cmo, "rocr": next_rocr, "ppo": next_ppo})
    
    return (rtn_pd, next_pd, scaler_dic)

In [371]:
val = etf_dic["0050"][["etf_close"]]
# print(val["etf_close"].tolist())
from sklearn.preprocessing import StandardScaler
import itertools
scaler = StandardScaler()
scaler.fit_transform(val)
#merged = list(itertools.chain.from_iterable(scaler.fit_transform(val)))
#merged
scaler.inverse_transform([-1.44880072])[0]

54.41707893989108

In [372]:
#create feature pandas
etf_pd_dic = {}
next_pd_dic = {}
scaler_dic = {}
for etfid in etf_ids:
    etf_pd, next_pd, scalers = create_feature(etf_dic[etfid.strip()], predict_start_date, {})
    etf_pd_dic.update({etfid.strip() : etf_pd})
    next_pd_dic.update({etfid.strip() : next_pd})
    scaler_dic.update(scalers)

In [373]:
etf_pd_dic["0050"]['etf_close'].describe()

count    1080.000000
mean       70.045093
std         7.961666
min        55.800000
25%        63.837500
50%        68.975000
75%        74.962500
max        88.300000
Name: etf_close, dtype: float64

In [374]:
#etf_pd_dic["0054"].describe()

In [375]:
#etf_pd_dic["00692"].describe()

In [376]:
from sklearn.preprocessing import MinMaxScaler
import itertools
# pd_0050=etf_pd_dic["0051"]
# pd_0050=pd_0050[pd_0050.etf_date > '20180101']
# scaler = MinMaxScaler()
# pd_0050['close_dif']=list(itertools.chain.from_iterable(scaler.fit_transform(pd_0050[['close_dif']])))
# # pd_0050[['close_dif']].plot()
# for col in list(pd_0050.columns.values): #["bias_3_6", "ema5_slope", 'ema10_slope']:
#     pd_0050[col]=list(itertools.chain.from_iterable(scaler.fit_transform(pd_0050[[col]])))
#     pd_0050[['close_dif',col]].plot()
# plt.show()

In [377]:
non_feature_list = ['etf_id', 'etf_date', 'close_dif', 'etf_close', 'idx']
feature_cols = [col for col in list(etf_pd_dic["0050"].columns.values) if col not in non_feature_list]
# feature_cols = ['ema5']
print(feature_cols)

['bias_3_6', 'bias_ma', 'cmo', 'd', 'dif', 'ema10', 'ema10_slope', 'ema20', 'ema3', 'ema30', 'ema3_slope', 'ema5', 'ema5_slope', 'k', 'macd1', 'macdhist1', 'macdsignal1', 'nbias20', 'nbias3', 'nbias6', 'ppo', 'rocr', 'rsi']


In [378]:
train_dt_org = pd.concat([etf_pd_dic[etfid.strip()] for etfid in etf_ids])
std_dic = {}
for col in [c for c in train_dt_org.columns.values if c not in ["etf_id", "etf_date"]]:
        scaler_key = col
        if scaler_key in scaler_dic:
            scaler = scaler_dic[scaler_key]
            merged = list(itertools.chain.from_iterable(scaler.transform(train_dt_org[[col]])))                
        else:
            scaler = MinMaxScaler()
            merged = list(itertools.chain.from_iterable(scaler.fit_transform(train_dt_org[[col]])))
            #保留etf_close的scaler在後續使用
            scaler_dic.update({scaler_key: scaler})
        std_dic.update({col: merged})
train_dt = pd.DataFrame(std_dic)
print('train_dt count:', train_dt.shape)

for etfid in etf_ids:
    std_dic = {}
    next_pd = next_pd_dic[etfid.strip()]
    for col in [c for c in next_pd.columns.values if c not in ["etf_id", "etf_date"]]:
        scaler = scaler_dic[col]
        merged = list(itertools.chain.from_iterable(scaler.transform(next_pd[[col]])))
        std_dic.update({col: merged})
    next_pd_dic.update({etfid.strip() : pd.DataFrame(std_dic)})
print('next_pd_dic count:', len(next_pd_dic.keys()), '*' , next_pd_dic['0050'].shape)

train_dt count: (15891, 26)
next_pd_dic count: 18 * (1, 26)


In [379]:
list('bias_3_6,ema20,ema30,macd1'.split(','))

['bias_3_6', 'ema20', 'ema30', 'macd1']

In [380]:
import xgboost as xgb
def trainModel(train_dt, feature_cols, f_type):
    train_x1 = train_dt[feature_cols].values
    if f_type == '1':
        #model for close_dif
        train_y1 = train_dt["close_dif"].values
        #$$$ parameter
        model1 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
            colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0 )
        model1.fit(train_x1, train_y1)
        return model1
    else:
        #model for etf_close
        train_y2 = train_dt["etf_close"].values
        model2 = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, 
            colsample_bytree=1, max_depth=68, n_jobs=4, min_child_weight=0 )
        model2.fit(train_x1, train_y2)
        return model2

In [381]:
pd_0050=etf_dic["0050"]
prev_date = pd_0050.loc[~pd_0050["etf_date"].isin(next_date_range)]["etf_date"].max()
prev_date

'20180601'

In [382]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#計算上或下的值(udf)
def judge_up_down_pred_native(curr_price, prev_close_price):
    if checkNan(prev_close_price):
        return 0.0
    else:
        prev_price = prev_close_price
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred_native, DoubleType())

In [383]:
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
#eval dt for evluate score
tetf_dt_eval = tetfp_dt.filter(tetfp_dt.etf_date.isin([prev_date] + next_date_range)) \
    .withColumn("prev_close", lag(tetfp_dt.etf_close).over(wsSpec_etf)) 
tetf_dt_eval =  tetf_dt_eval.withColumn("act_ud", judge_up_down_pred(tetf_dt_eval.etf_close, tetf_dt_eval.prev_close))
tetf_dt_eval = tetf_dt_eval.filter(tetf_dt_eval.etf_date.isin(next_date_range))
tetf_dt_eval.cache()
tetf_dt_eval.show()

+-------+--------+----------------+--------+--------+-------+---------+---------+----------+------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|prev_close|act_ud|
+-------+--------+----------------+--------+--------+-------+---------+---------+----------+------+
|00701  |20180604|   國泰臺灣低波動30    |   21.14|   21.24|  21.14|    21.24|    140.0|     21.03|   1.0|
|00701  |20180605|   國泰臺灣低波動30    |    21.3|   21.33|  21.27|    21.32|     66.0|     21.24|   1.0|
|00701  |20180606|   國泰臺灣低波動30    |   21.36|   21.44|  21.36|    21.43|    163.0|     21.32|   1.0|
|00701  |20180607|   國泰臺灣低波動30    |   21.51|   21.53|  21.43|     21.5|     53.0|     21.43|   1.0|
|00701  |20180608|   國泰臺灣低波動30    |   21.42|   21.42|  21.34|    21.34|      7.0|      21.5|   2.0|
|0051   |20180604|元大中型100         |   34.13|   34.25|  34.13|    34.25|     36.0|     33.94|   1.0|
|0051   |20180605|元大中型100         |   34.35|    34.4|   34.3|    34.32|     12.0|     34.25|   1.0|


In [384]:
tetf_pd_eval = tetf_dt_eval.toPandas()
# tetf_pd_eval.loc[(tetf_pd_eval['etf_id'] == '006203 ') & (tetf_pd_eval['etf_date'] == '20180522')]['etf_close'].values[0]

In [385]:
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
def doEvaluate(predict_res_final, tetf_pd_eval):
    etf_close_list = []
    act_ud_list = []
    prediction_list = []
    pred_ud_list = []
    acc_list = []
    weights = [0.1, 0.15, 0.2, 0.25, 0.3]
    judge_score = 0.0
    for row in predict_res_final:
        eid = row['etf_id']
        edate = row['etf_date']
        if edate not in next_date_range:
            continue
        etf_eval = tetf_pd_eval.loc[(tetf_pd_eval['etf_id'] == eid) & (tetf_pd_eval['etf_date'] == edate)]
        etf_close = etf_eval['etf_close'].values[0]
        act_ud = etf_eval['act_ud'].values[0]
        prediction = row['prediction']
        pred_ud = row['pred_ud']
        #etf score
        eidx = next_date_range.index(edate)
        judge_score = judge_score + \
            ((0.5 if pred_ud == act_ud else 0.0) + \
             ((etf_close-abs(prediction-etf_close))/etf_close)*0.5)*weights[eidx]
        etf_close_list.append(etf_close)
        act_ud_list.append(act_ud)
        prediction_list.append(prediction)
        pred_ud_list.append(pred_ud)
        acc_list.append(1.0 if pred_ud == act_ud else 0.0)
    #evaluate accuracy_score
    accuracy = accuracy_score(act_ud_list, pred_ud_list, normalize=True)
#     print(act_ud_list)
#     print(pred_ud_list)
    rmse = mean_squared_error(etf_close_list, prediction_list)
    return (rmse, accuracy, judge_score)
    

In [386]:
scaler_dic.keys()

dict_keys(['bias_3_6', 'bias_ma', 'close_dif', 'cmo', 'd', 'dif', 'ema10', 'ema10_slope', 'ema20', 'ema3', 'ema30', 'ema3_slope', 'ema5', 'ema5_slope', 'etf_close', 'k', 'macd1', 'macdhist1', 'macdsignal1', 'nbias20', 'nbias3', 'nbias6', 'ppo', 'rocr', 'rsi', 'idx'])

In [387]:
def max_7percent(prev_close, prediction):
    dif = prediction - prev_close
    pct = 0.07
    dif_max = prev_close * pct
    rtn_val = prediction
    if abs(dif) > dif_max:
        if dif > 0:
            rtn_val = prev_close + dif_max
        else:
            rtn_val = prev_close - dif_max
    return rtn_val
        
def doPredict1(next_date_range, model1, etf_dic, next_pd_dic, scaler_dic, feature_cols):
    predict_res_final_1 = []
    dummy_date = '20999999'
    for date in next_date_range:
        for eid in etf_ids:
            #do predict
            pred_data = next_pd_dic[eid.strip()][feature_cols].values
            pred_res = model1.predict(data=pred_data)
#             print(pred_res)

            etf_dt = etf_dic[eid.strip()]
            prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
#             print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
            prev_close = prev_dt["etf_close"]
            scaler = scaler_dic['close_dif']
            prediction = prev_close + scaler.inverse_transform(pred_res[0])[0][0]
            prediction = max_7percent(prev_close, prediction)
            #取出所有預測結果作合併，以進行後續成績計算
            predict_res_final_1.append({
                'etf_id': eid, 'etf_date': date,
                'prediction': prediction, 'pred_ud': judge_up_down_pred_native(prediction, prev_close)
            })

            if (date != next_date_range[-1]) and (date != dummy_date):
                #作出新的next_pd
                new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                           'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
                new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                          'etf_low', 'etf_close', 'etf_count']]
                #print(new_pd)
                etf_dt=etf_dt.append(new_pd, ignore_index=True)
                next_date = next_date_range[next_date_range.index(date)+1]
                etf_pd2, next_pd2, _ = create_feature(etf_dt, next_date, scaler_dic)
                etf_dic.update({eid.strip(): etf_dt})
                next_pd_dic.update({eid.strip(): next_pd2})
    return predict_res_final_1
    
def doPredict2(next_date_range, model2, etf_dic, next_pd_dic, scaler_dic, feature_cols):
    predict_res_final_2 = []
    for date in next_date_range:
        for eid in etf_ids:
            #do predict
            pred_data = next_pd_dic[eid.strip()][feature_cols].values
            pred_res = model2.predict(data=pred_data)
#             print(eid.strip(), '-', date, ': ', pred_res[0], '-', 
#                   scaler_dic[eid.strip()+'-etf_close'].inverse_transform(pred_res[0])[0][0])

            etf_dt = etf_dic[eid.strip()]
            prev_dt = etf_dt.loc[(etf_dt["etf_date"] < date) ].iloc[-1]
    #         print('id of prev_dt: ', eid, ', date: ' ,prev_dt["etf_date"])
            prev_close = prev_dt["etf_close"]
            scaler = scaler_dic['etf_close']
            prediction = scaler.inverse_transform(pred_res[0])[0][0]
            prediction = max_7percent(prev_close, prediction)
            #取出所有預測結果作合併，以進行後續成績計算
            predict_res_final_2.append({
                'etf_id': eid, 'etf_date': date,
                'prediction': prediction, 'pred_ud': judge_up_down_pred_native(prediction, prev_close)
            })

            if date != next_date_range[-1]:
                #作出新的next_pd
                new_dic = {'etf_id': [eid], 'etf_date': [date], 'etf_name': [''], 
                           'etf_open': [0.0],  'etf_high': [0.0], 'etf_low': [0.0], 'etf_close': [prediction], 'etf_count': [0.0]}
                new_pd = pd.DataFrame(data=new_dic)[['etf_id','etf_date','etf_name', 'etf_open', 'etf_high', 
                                                          'etf_low', 'etf_close', 'etf_count']]
                #print(new_pd)
                etf_dt=etf_dt.append(new_pd, ignore_index=True)
                next_date = next_date_range[next_date_range.index(date)+1]
                etf_pd2, next_pd2, _ = create_feature(etf_dt, next_date, scaler_dic)
                etf_dic.update({eid.strip(): etf_dt})
                next_pd_dic.update({eid.strip(): next_pd2})
    return predict_res_final_2

In [388]:
# for subset in itertools.combinations(feature_cols, 1):
#     print(list(subset))

In [ ]:
feature_dic = {
    'ema5,dif':'1',
    'rocr,ema5':'2',
    'ema20,d':'1',
    'ema5,k':'2',
    'ema5_slope,k':'1',
    'ema5_slope,ema10':'1',
    'ema20,nbias6':'2',
    'ema30,d':'1',
    'dif,ema3':'1',
    'ema20,rocr':'2',
    'nbias3,ema20':'2',
    'nbias6,ema3_slope':'1',
    'cmo,bias_3_6':'1',
    'ema5,rsi':'2',
    'ema3,k':'1'
}

feature_dic = {
    'ema20,nbias6,macd1':'2',
    'dif,ema30,nbias6':'2',
    'ema20,dif,ema5_slope':'2',
    'ema3,k,d':'2',
    'dif,ema3_slope,ema10':'2',
    'ema5,dif,nbias3':'1',
    'ema20,d,macdsignal1':'2',
    'cmo,bias_3_6,macd1':'1'
}

In [ ]:
type_list = []
feature_list = []
acc_list = []
rmse_list = []
escore_list = []
all_pred_list = []

# for features in itertools.combinations(feature_cols, feature_num):
for feature_str in feature_dic.keys():    
    f_type = feature_dic[feature_str]
    feature_cols = list(feature_str.split(','))
    model = trainModel(train_dt, feature_cols, f_type)
    #recover to re-run
    for eid in etf_ids:
        etf_dt = etf_dic[eid.strip()]
        etf_dt = etf_dt.loc[~etf_dt["etf_date"].isin(next_date_range)]
        etf_dic.update({eid.strip(): etf_dt})
        etf_pd, next_pd, _ = create_feature(etf_dic[eid.strip()], predict_start_date, scaler_dic)
#         print(eid.strip(), ': ', etf_pd.shape)
        etf_pd_dic.update({eid.strip() : etf_pd})
        next_pd_dic.update({eid.strip() : next_pd})
    if f_type == '1':
        #evaluate close_dif
        predict_res_pd_1 = doPredict1(next_date_range, model, etf_dic, next_pd_dic, scaler_dic, feature_cols)
        all_pred_list = all_pred_list + predict_res_pd_1
        if DoEval:
            rmse1, acc1, e_score1 = doEvaluate(predict_res_pd_1, tetf_pd_eval)
            type_list.append("1")
            feature_list.append(feature_str)
            acc_list.append(acc1)
            rmse_list.append(rmse1)
            escore_list.append(e_score1)
        if DoEval:
            print(feature_str, ': final1=', e_score1)
    else:
        #evaluate etf_close
        predict_res_pd_2 = doPredict2(next_date_range, model, etf_dic, next_pd_dic, scaler_dic, feature_cols)
        all_pred_list = all_pred_list + predict_res_pd_2
        if DoEval:
            rmse2, acc2, e_score2 = doEvaluate(predict_res_pd_2, tetf_pd_eval)
            type_list.append("2")
            feature_list.append(feature_str)
            acc_list.append(acc2)
            rmse_list.append(rmse2)
            escore_list.append(e_score2)
        if DoEval:
            print(feature_str, ': final2=', e_score2)

score_dic = {
    'feature_type' : type_list, 'feature': feature_list, 'accuracy': acc_list, 'rmse': rmse_list,
    'final_score': escore_list
}
if DoEval:
    score_dic = {
        'feature_type' : type_list, 'feature': feature_list, 'accuracy': acc_list, 'rmse': rmse_list,
        'final_score': escore_list
    }


ema20,nbias6,macd1 : final2= 14.898895412857804
dif,ema30,nbias6 : final2= 14.889072084305246
ema20,dif,ema5_slope : final2= 14.844486638603959
ema3,k,d : final2= 14.797666048558014
dif,ema3_slope,ema10 : final2= 14.603483468527063
ema5,dif,nbias3 : final1= 14.57081352054886


In [ ]:
# etf_dic['0050']
# score_dic['feature']

In [ ]:
score_pd = pd.DataFrame(data=score_dic)[['feature_type', 'feature', 'accuracy', 'rmse', 'final_score']]

In [ ]:
score_pd.sort_values(by=['final_score'], ascending=[0]).head(20)

In [ ]:
all_pred_pd = pd.DataFrame(all_pred_list)

In [ ]:
def judge_ud(ud_list):
    num_1 = 0
    num_0 = 0
    num_2 = 0
    for ud in ud_list:
        if ud == 1.0:
            num_1 += 1
        elif ud == 2.0:
            num_2 += 1
        else:
            num_0 += 1
    if num_2 >= num_1 and num_2 >= num_0:
        return 2.0
    if num_1 >= num_2 and num_1 >= num_0:
        return 1.0
    return 0.0
    
predict_res_final = []
for date in next_date_range:
    for eid in etf_ids:
        pred_pd = all_pred_pd.loc[(all_pred_pd['etf_id'] == eid) & (all_pred_pd['etf_date'] == date)]
        ud_list = pred_pd['pred_ud'].tolist()
        ud = judge_ud(ud_list)
        close_list = pred_pd["prediction"].tolist()
        close = avg_list(close_list)
        predict_res_final.append({
            'etf_id': eid, 'etf_date': date,
            'prediction': close, 'pred_ud': ud
        })

if DoEval:
    rmse, acc, e_score = doEvaluate(predict_res_final, tetf_pd_eval)
    print(predict_start_date, ' final: rmse=', rmse, ', accuracy=', acc, 'etf_score=', e_score)
# 20180604  final: rmse= 0.35411602595914005 , accuracy= 0.7666666666666667 etf_score= 15.684756862674414
# 20180528  final: rmse= 0.21377914756013033 , accuracy= 0.4444444444444444 etf_score= 12.73777330635146
# 20180521  final: rmse= 0.4968230593196396 , accuracy= 0.45555555555555555 etf_score= 13.644212655410296
# 20180514  final: rmse= 0.3106272364914174 , accuracy= 0.6 etf_score= 14.756825799674976
# 20180507  final: rmse= 0.4034365973828622 , accuracy= 0.5111111111111111 etf_score= 13.246689204250412

In [ ]:
final_res = spark.createDataFrame(pd.DataFrame(predict_res_final)).orderBy("etf_id", "etf_date").collect()
#export to pandas
etf_ids = []
mon_ud = []
mon_price = [] 
tue_ud = []
tue_price = []
wed_ud = []
wed_price = []
thu_ud = []
thu_price = []
fri_ud = []
fri_price = []

def encode_ud(oper_ud):
    if oper_ud == 0.0:
        return 0
    elif oper_ud == 1.0:
        return 1
    else:
        return -1

for row in final_res:
    etf_id = row["etf_id"]
    if etf_id not in etf_ids:
        etf_ids.append(etf_id)
    etf_date = row["etf_date"]
    eidx = next_date_range.index(row["etf_date"])
    if eidx == 0:
        mon_ud.append(encode_ud(row["pred_ud"]))
        mon_price.append(row["prediction"])
    elif eidx == 1:
        tue_ud.append(encode_ud(row["pred_ud"]))
        tue_price.append(row["prediction"])
    elif eidx == 2:
        wed_ud.append(encode_ud(row["pred_ud"]))
        wed_price.append(row["prediction"])
    elif eidx == 3:
        thu_ud.append(encode_ud(row["pred_ud"]))
        thu_price.append(row["prediction"])
    elif eidx == 4:
        fri_ud.append(encode_ud(row["pred_ud"]))
        fri_price.append(row["prediction"])
        
if len(mon_ud) == 0:
    mon_ud = list(0 for i in range(0,len(etf_ids)))
    mon_price = list(0.0 for i in range(0,len(etf_ids)))
if len(tue_ud) == 0:
    tue_ud = list(0 for i in range(0,len(etf_ids)))
    tue_price = list(0.0 for i in range(0,len(etf_ids)))
if len(wed_ud) == 0:
    wed_ud = list(0 for i in range(0,len(etf_ids)))
    wed_price = list(0.0 for i in range(0,len(etf_ids)))
if len(thu_ud) == 0:
    thu_ud = list(0 for i in range(0,len(etf_ids)))
    thu_price = list(0.0 for i in range(0,len(etf_ids)))
if len(fri_ud) == 0:
    fri_ud = list(0 for i in range(0,len(etf_ids)))
    fri_price = list(0.0 for i in range(0,len(etf_ids)))
    
dic = {"ETFid": etf_ids, 
       "Mon_ud": mon_ud, "Mon_cprice": mon_price,
       "Tue_ud": tue_ud, "Tue_cprice": tue_price,
       "Wed_ud": wed_ud, "Wed_cprice": wed_price,
       "Thu_ud": thu_ud, "Thu_cprice": thu_price,
       "Fri_ud": fri_ud, "Fri_cprice": fri_price
      }
final_df = pd.DataFrame(data=dic)[['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice',
                                  'Wed_ud','Wed_cprice','Thu_ud','Thu_cprice',
                                  'Fri_ud','Fri_cprice']]
final_df

In [ ]:
final_df.to_csv(Path.replace("file:","") + "/etf_price_pred_talib_eval_mac.csv",index=False)